In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
pd.read_csv?

In [ ]:
!ls ../input/male-daan-schnell-mal-klassifizieren

In [ ]:
!ls ../input/male-daan-schnell-mal-klassifizieren/train.csv

In [ ]:
!head ../input/male-daan-schnell-mal-klassifizieren/train.csv

In [ ]:
!dir ../input/male-daan-schnell-mal-klassifizieren

![](http://)Um eine .csv-Datei zu laden, benutzen wir Pandas, eine Bibliothek für Data Science. Wir laden ein *DataFrame* (eine Tabelle mit benannten Spalten), werden anschliessend aber nur den Numpy-Array verwenden, welcher unter dem Attribut df.values abgespeichert ist. 

In [ ]:
fn = '../input/male-daan-schnell-mal-klassifizieren/train.csv'
import pandas as pd
df = pd.read_csv(fn)
df.head(5)

In [ ]:
df = pd.read_csv(fn,index_col='Id')
df.head()

In [ ]:
df.y.values.dtype

In [ ]:
df.values

In [ ]:
data = df.values
data

Numpy enthält eine n-dimensionale Matrix (hier tatsächlich eine Matrix, n=2), wobei alle Matrixelemente vom selben Typ sind (hier 'float64', also Gleitkommazahl mit 64 Bit, in C wäre dies der Typ 'double'):

In [ ]:
data.shape,data.dtype

benutzen wir eine etwas passendere Notation: Unsere Daten sollten wir eigentlich in Trainings- und Testdaten aufteilen. Hier machen wir's uns (methodisch ein grosser Fehler!) der Einfachheit halber- einfach: Die ersten *nTrain* Zeilen sollen Trainingsdaten sein, der Rest Testdaten.
Aber: die letzte Spalte enthält das Klassifikationslabel- diese gliedern wir in ein Symbol $y$ aus: $f(X)\sim y$

In [ ]:
nTrain=5000
Xtrain = data[:nTrain,:-1] 
ytrain = data[:nTrain,-1] 
Xtest = data[nTrain:,:-1] 
ytest = data[nTrain:,-1] 
Xtrain.shape,ytrain.shape,Xtest.shape,ytest.shape

In [ ]:
ytrain

Besser wäre es, wenn `ytrain` eine Ganzzahl wäre: 

In [ ]:
ytrain = ytrain.astype('int')
ytest = ytest.astype('int')
ytrain, ytrain.dtype

Bauen wir einen k-NN Klassifikator für folgendes $k$.

In [ ]:
Xtrain

In [ ]:
k=7

Das Training ist trivial! Wir müssen nur die Trainingsdaten abspeichern (schon geschehen). 
Für die Vorhersage: Wir beginnen mit der Vorhersage für eine Testdatenzeile, welche wir zufällig auswählen:

In [ ]:
#np.random.randint? #Tipp: entkommentieren Sie diese Zeile und lernen Sie, was die Funktion randint ist.
zufälliger_Index = np.random.randint(low=0,high=len(ytest))
zufälliger_Index

In [ ]:
Testzeile = Xtest[zufälliger_Index,:]
Testlabel = ytest[zufälliger_Index]
Testzeile,Testlabel

Für `Testzeile` benötigen wir die `k` kleinsten Abstände zu Trainingsdatenpunkten (in `Xtrain`). Diese zu berechnen ist eine der Stärken von Numpy: Mit einer Zeile berechnen wir alle Abstände der Testzeile zu *allen* Trainingszeilen. Es lohnt sich, über *broadcasting* nachzulesen, aber grundsätzlich: Alle Numpy-Operationen werden Elementweise durchgeführt. Die Funktion np.sum und viele weitere haben ein Argument axis=..., mit welchem sich angeben lässt, worüber summiert wird: None (über alles), 0 (zeilenweise), 1 (spaltenweise), 2 etc.

In [ ]:
Xtrain.shape, Testzeile.shape

In [ ]:
((Xtrain - Testzeile)**2)

In [ ]:
np.sqrt?

In [ ]:
distanz = np.sqrt(((Xtrain - Testzeile)**2).sum(axis=1))

In [ ]:
distanz

Für die Differenz wird die Testzeile so oft repliziert, damit die entstehende Shape mit jener von Xtrain kompatibel ist für elementweises Subtrahieren. Auch das Quadrieren ("\*\*2") wird Elementweise ausgeführt. Hier wird also 
$$\sqrt{(xt_1-x_1)^2+(xt_2-x_2)^2} \qquad\forall\, xt \mathrm{~in~} Xtest$$
(mit $xt$ einer Zeile aus dem Trainingsdatensatz, und x der `Testzeile`)

In [ ]:
distanz

In [ ]:
np.sort(distanz)[:k]

In [ ]:
np.argsort(distanz)[:k]

In [ ]:
np.argsort([1,7,4,9])

In [ ]:
ytrain

In [ ]:
distanz[np.argsort(distanz)[:k]]
np.min(distanz)

In [ ]:
ytrain[np.argsort(distanz)[:k]]

In [ ]:
[1,0,1,0,1,1,0]

In [ ]:
np.around(np.mean(np.array([1,0,1,0,1,1,0])))

In [ ]:
np.median([1,0,1,0,1,1,0])

In [ ]:
distanz[:k]

nun brauchen wir "nur" noch die Labels der $k$ kleinsten Distanzen. Dazu
- berechnen wir die Indices der $k$ kleinsten Distanzen mittels np.argsort
- suchen uns an Hand dieser Indices in `ytrain` die entsprechenden Labels der nächsten Trainingsdatenpunkte
- Bestimmen darunter das häufigere Label. Dieses ist dann unsere k-NN-Vorhersage für die `Testzeile`.
    

## 1. berechne die Indices der $k$ kleinsten Distanzen mittels np.argsort

Um zu verstehen, wie argsort funktioniert:

In [ ]:
a= np.array([1,4,8,2])
sorted_indices = np.argsort(a)
sorted_indices, a[sorted_indices]


In [ ]:
sorted_indices = np.argsort(distanz)
distanz[sorted_indices]

## suchen an Hand dieser Indices in `ytrain` die entsprechenden Labels der nächsten Trainingsdatenpunkte

Damit lassen sich die $k$ kleinsten Distanzen, deren Indices und die zugehörigen Testlabels ausgeben:

In [ ]:
distanz[sorted_indices[:k]], sorted_indices[:k], ytrain[sorted_indices[:k]]

In [ ]:
Auftretende_Trainingslabels = ytrain[sorted_indices[:k]]

## Bestimmen darunter das häufigere Label. Dieses ist dann unsere k-NN-Vorhersage für die `Testzeile`.

Welches dieser Label ist das häufigste? Da nur die Werte 0 und 1 vorkommen, können wir dazu den Mittelwert bestimmen: Ist er grösser als 0.5, dann kommt 1 häufiger vor als 0.

In [ ]:
if np.mean(Auftretende_Trainingslabels)>=0.5:
    yhat = 1
else:
    yhat = 0
yhat

Etwas eleganter wäre die folgende Zeile. Bincount gibt einen Array zurück, der an der 0. Stelle die Anzahl Nullen, an der 1. Stelle die Anzahl Einsen (etc.) wiedergibt. Also aus [1,0,1,0,0] wird [3,2]. ArgMax gibt anschliessend den Index der Stelle raus, welche das (erste) Maximum enthält).

In [ ]:
print('Häufigstes Label in [1,0,1,0,0]:',np.argmax(np.bincount([1,0,1,0,0])))
print('Häufigstes Label in [1,0,1,0,1]:',np.argmax(np.bincount([1,0,1,0,1])))

Zusammenfassend:

In [ ]:
np.bincount([0,3,3,2,1,1])

In [ ]:
np.bincount([0,3,3,3,2,2,2,2,2,2,2,1,1,4,4,4,4,5,6])

In [ ]:
np.argmax(np.bincount([0,3,3,3,2,2,2,2,2,2,2,1,1,4,4,4,4,5,6]))

In [ ]:
def kNN_Vorhersage(Xtrain,Testzeile,k):
    distanz = (((Xtrain - Testzeile)**2).sum(axis=1))**0.5
    sorted_indices = np.argsort(distanz)
    Auftretende_Trainingslabels = ytrain[sorted_indices[:k]]
    return np.argmax(np.bincount(Auftretende_Trainingslabels))

zufälliger_Index = np.random.randint(low=0,high=len(ytest))
Testzeile = Xtest[zufälliger_Index,:]

yhat = kNN_Vorhersage(Xtrain,Testzeile,k)

print(f'Vorhersage für {Testzeile}:{yhat}')

# k-NN mit Scikit-Learn
Natürlich geht das alles einfacher mit Scikit-Learn! Doku zum KNN [hier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [ ]:
#kNN-Klassifikator in Scikit-Learn
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5)
#clf = KNeighborsClassifier(n_neighbors=k)
#KNeighborsClassifier? #Bitte entkommentieren!

In [ ]:
#Laden der Daten:
import pandas as pd
df_train = pd.read_csv('../input/male-daan-schnell-mal-klassifizieren/train.csv',index_col='Id')
df_test = pd.read_csv('../input/male-daan-schnell-mal-klassifizieren/test.csv',index_col='Id')
df_train.head(3)

In [ ]:
# Bringe Daten in X-y-Form:
Xtrain = df_train.values[:,:-1]
ytrain = df_train.values[:,-1]
Xtest = df_test.values

In [ ]:
#Trainiere den k-NN:
clf.fit(Xtrain,ytrain)

In [ ]:
yhat = clf.predict(Xtest)

In [ ]:
#Erstelle eine .csv-Datei "Submission.csv"
ser = pd.Series(yhat,name='y').astype('int')
ser.index.name='Id'
ser.to_csv('Submission.csv',header=True)
!head Submission.csv

Dieses Notebook laden wir nun als Eingabe für die Kaggle-Competition hoch! Dazu wird das Notebook gespeichert. Unter Output findet man dann den Knopf "Submit to Competition"